## Creation d'une BDD originale

In [6]:
import sqlite3
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker('fr_FR')

# Connexion à la base de données SQLite
connexion = sqlite3.connect('PaddlePassion.db')
cursor = connexion.cursor()

# Création de la table des employés
cursor.execute('''CREATE TABLE employes (
                    id_em INTEGER PRIMARY KEY,
                    nom TEXT,
                    genre TEXT,
                    date_naissance DATE,
                    date_embauche DATE,
                    date_depart DATE,
                    id_li INTEGER,
                    FOREIGN KEY (id_li) REFERENCES lieux (id_li)
                )''')

# Création de la table des lieux de travail
cursor.execute('''CREATE TABLE lieux (
                    id_li INTEGER PRIMARY KEY,
                    ville TEXT
                )''')

# Génération des données pour la table des employés
for _ in range(20):
    nom = fake.last_name()
    genre = random.choice(['Homme', 'Femme'])
    date_naissance = fake.date_of_birth(minimum_age=23, maximum_age=62).strftime('%Y-%m-%d')
    date_embauche = fake.date_between(start_date='-40y', end_date='-1d').strftime('%Y-%m-%d')
    
    date_depart = random.choice(['9999-01-01', None])
    if date_depart != '9999-01-01':
        date_embauche_dt = datetime.strptime(date_embauche, '%Y-%m-%d')
        date_depart_dt = fake.date_between_dates(date_start=date_embauche_dt + timedelta(days=1),
                                                 date_end=datetime.today()).strftime('%Y-%m-%d')
        date_depart = date_depart_dt
    
    id_li = _ + 1  # id_li est égal à id_em associé
    

    # Insertion des données dans la table des employés
    cursor.execute('''INSERT INTO employes (nom, genre, date_naissance, date_embauche, date_depart, id_li)
                      VALUES (?, ?, ?, ?, ?, ?)''', (nom, genre, date_naissance, date_embauche, date_depart, id_li))

# Génération des données pour la table des lieux de travail
for ville in range(20):
    ville = random.choice(['Orléans', 'Blois', 'Tours', 'Bourges'])    # Je peux sans doute remonter tout ça dans la boucle du dessus
    # Insertion des données dans la table des lieux de travail
    cursor.execute('''INSERT INTO lieux (ville)
                      VALUES (?)''', (ville,))

# Validation et fermeture de la connexion à la base de données
connexion.commit()
connexion.close()


## Saisie des données par un utilisateur

In [ ]:
from datetime import datetime, timedelta

def validate_date(date_string):
    try:
        date = datetime.strptime(date_string, "%Y-%m-%d")
        return date
    except ValueError:
        return None


def inserer_informations_nouvel_employe() :
    
    genre_possible = ["Homme", "Femme"]
    ville_possible = ['Orléans', 'Blois', 'Tours', 'Bourges']

    nom = input("Quel est le nom de l'employé: ") 
    
    while genre not in genre_possible :
        print(f"La valeur saisie n'est pas correcte : {genre}.")
        print("Merci de saisir une des valeurs suivantes : ", end="")
        print(", ".join(genre_possible))
        genre =input("Quelle est le genre de l'employé : ")

    date_naissance = None
    while date_naissance is None:
        date_naissance = validate_date(input("Date de naissance (Format: yyyy-mm-dd) : "))
        if date_naissance is None:
            print("La date saisie est invalide. Veuillez réessayer.")
    
    # Validation de la date d'embauche
    date_embauche = None
    while date_embauche is None:
        date_embauche = validate_date(input("Date d'embauche (Format: yyyy-mm-dd) : "))
        if date_embauche is None:
            print("La date saisie est invalide. Veuillez réessayer.")
    
    # Validation de la date de départ
    date_depart = None
    while date_depart is None:
        date_depart_str = input("Date de départ (Format: yyyy-mm-dd ou 9999-01-01 pour aucune date de départ) : ")
        if date_depart_str == '9999-01-01':
            date_depart = date_depart_str
        else:
            date_depart = validate_date(date_depart_str)
            if date_depart is None or date_depart <= date_embauche:
                print("La date saisie est invalide. Veuillez réessayer.")

    #  Validation de la ville            
    while ville not in ville_possible :
        print(f"La valeur saisie n'est pas correcte : {ville}.")
        print("Merci de saisir une des valeurs suivantes : ", end="")
        print(", ".join(ville_possible))
        genre =input("Ville d'affectation de l'employé : ")

    employe_data = [nom, genre, date_naissance, date_embauche, date_depart]
    ville_data =[ville]
    
    return employe_data
    return ville_data

encore_une_saisie = "o"

while encore_une_saisie.lower() != "n":
    employe_data = inserer_informations_nouvel_employe()
    print(employe_data)
    
    # Insérer les données dans la base de données ici

    try : 
        cursor.execute("INSERT INTO employes VALUES(?,?,?,?,?)", employe_data)
    except sqlite3.IntegrityError as e:
        print(e)
    
    encore_une_saisie = input("Voulez-vous continuer (o/n) : ")